<a href="https://colab.research.google.com/github/matiasvallejosdev/ar-covid-interactive-visualizer/blob/WorldVisualizer/Colab_5_Construir_RNR_Valoraciones_de_Google_Play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifier to google play reviews

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as tfl

import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## 1. Load data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataframe = pd.read_csv('/content/drive/My Drive/Machine Learning/Datasets/googleplaystore_reviews/googleplaystore_user_reviews.csv')

In [ ]:
dataframe = dataframe.dropna()
dataframe.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000
5,10 Best Foods for You,Best way,Positive,1.00,0.300000


In [ ]:
dataframe.describe()

,Sentiment_Polarity,Sentiment_Subjectivity
count,37427.000000,37427.000000
mean,0.182171,0.492770
std,0.351318,0.259904
min,-1.000000,0.000000
25%,0.000000,0.357143
50%,0.150000,0.514286
75%,0.400000,0.650000
max,1.000000,1.000000


## 2. Preprocessing data

In [ ]:
dataframe = dataframe[['Translated_Review','Sentiment']]
dataframe.head()

,Translated_Review,Sentiment
0,I like eat delicious food. That's I'm cooking ...,Positive
1,This help eating healthy exercise regular basis,Positive
3,Works great especially going grocery store,Positive
4,Best idea us,Positive
5,Best way,Positive


In [ ]:
def preprocess_text(sen):
    # Eliminar símbolos de puntuación y números
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Eliminar carácteres sueltos
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Eliminar espacios excesivos
    sentence = re.sub(r'\s+', ' ', sentence)

    # Convertir a minúscula
    sentence = sentence.lower()
  
    # Eliminar las stopwords.
    words = sentence.split()
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    
    return ' '.join(filtered_words)

In [ ]:
dataframe['Translated_Review'] = dataframe['Translated_Review'].apply(lambda sen: preprocess_text(sen))
dataframe

,Translated_Review,Sentiment
0,like eat delicious food cooking food case best...,Positive
1,help eating healthy exercise regular basis,Positive
3,works great especially going grocery store,Positive
4,best idea us,Positive
5,best way,Positive
...,...,...
64222,ads older many agents much owner posted detail...,Positive
64223,photos posted portal load fit purpose sure sto...,Positive
64226,dumb app wanted post property rent give option...,Negative
64227,property business got link sms happy performan...,Positive


In [ ]:
def determine_class(label):
  if label == 'Positive':
    return 0
  elif label == 'Neutral':
    return 1
  elif label == 'Negative':
    return 2

REMOVE_NEUTRAL = False
MERGE_NEGATIVE_NEUTRAL = False

if REMOVE_NEUTRAL:
  indexNames = dataframe[dataframe['Sentiment'] == 'Neutral'].index
  dataframe.drop(indexNames , inplace=True)

  y = dataframe['Sentiment'].apply(lambda x: 1 if x == 'Positive' else 0).to_numpy()
  #y = tf.one_hot(y, 2)
else:
  if MERGE_NEGATIVE_NEUTRAL:
    y = dataframe['Sentiment'].apply(lambda x: 1 if x == 'Positive' else 0).to_numpy()
    #y = tf.one_hot(y, 2)
  else: 
    y = dataframe['Sentiment'].apply(lambda x: determine_class(x)).to_numpy()
    #y = tf.one_hot(y, 3)

X = dataframe['Translated_Review']
y = y.astype(np.uint8)

In [ ]:
print('The shape of X is:', X.shape)
print('The shape of y is:', y.shape)

The shape of X is: (37427,)
The shape of y is: (37427,)


In [ ]:
X.head()

0    like eat delicious food cooking food case best...
1           help eating healthy exercise regular basis
3           works great especially going grocery store
4                                         best idea us
5                                             best way
Name: Translated_Review, dtype: object

In [ ]:
y

array([0, 0, 0, ..., 2, 0, 2], dtype=uint8)

Split data to train and test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

Padding frecuent preproccesing data

In [ ]:
NUMBER_OF_WORDS = 20000
MAX_LEN = 100

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = NUMBER_OF_WORDS)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=MAX_LEN)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=MAX_LEN)

## 3. Create the model

In [ ]:
VOCABULARY_SIZE = NUMBER_OF_WORDS
EMBEDDING_SIZE = 128

In [ ]:
model = tf.keras.Sequential([
                             tfl.Embedding(VOCABULARY_SIZE, EMBEDDING_SIZE, input_shape=(X_train.shape[1],)),
                             tfl.LSTM(units=128, activation='tanh'),
                             tfl.Dense(64, activation='relu'),
                             tfl.Dropout(0.2),
                             tfl.Dense(units=np.unique(y_train).shape[0], activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
dropout_11 (Dropout)         (None, 100, 128)          0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 387       
Total params: 2,691,971
Trainable params: 2,691,971
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [ ]:
EPOCHS = 10
BATCH_SIZE = 128

In [ ]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/10
234/234 [==============================] - 93s 388ms/step - loss: 0.9023 - sparse_categorical_accuracy: 0.6395
Epoch 2/10
234/234 [==============================] - 90s 386ms/step - loss: 0.8960 - sparse_categorical_accuracy: 0.6417
Epoch 3/10
234/234 [==============================] - 91s 387ms/step - loss: 0.8953 - sparse_categorical_accuracy: 0.6417
Epoch 4/10
234/234 [==============================] - 90s 386ms/step - loss: 0.8948 - sparse_categorical_accuracy: 0.6418
Epoch 5/10
234/234 [==============================] - 91s 389ms/step - loss: 0.8940 - sparse_categorical_accuracy: 0.6418
Epoch 6/10
234/234 [==============================] - 91s 389ms/step - loss: 0.8919 - sparse_categorical_accuracy: 0.6416
Epoch 7/10
234/234 [==============================] - 91s 388ms/step - loss: 0.8806 - sparse_categorical_accuracy: 0.6452
Epoch 8/10
234/234 [==============================] - 90s 386ms/step - loss: 0.8847 - sparse_categorical_accuracy: 0.6488
Epoch 9/10
234/234 [====

In [ ]:
test_loss, test_acurracy = model.evaluate(X_test, y_test)
print("Test accuracy: {}".format(test_acurracy))

234/234 [==============================] - 8s 33ms/step - loss: 0.8526 - sparse_categorical_accuracy: 0.6392
Test accuracy: 0.6391931772232056
